In [1]:
!pip install python-dotenv
!pip install transformers
!pip install bitsandbytes
!pip install accelerate
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 2.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

### **練習 #1**

使用 Extractive QA model 以 `pipeline` 做 question-answering 任務：
- **給定文本**：the name of repo is bert-base-uncased
- **問題目標**：問模型 repo 的名稱
- **預期答案**：bert-base-uncased

In [2]:
# TODO
# practice 1 不需要特別指定模型，pipeline 預設載入 distilbert-base-cased-distilled-squad,
# 其為 Extractive QA 類摘要模型

from transformers import pipeline

利用 Hugging Face 的 text-generation model 與 Sentence Transformers embedding model 實作 QA 檢索聊天機器人。</p>
基於提供的資料集，使用 Embedding Cosine Similarity 檢索參考資料，再透過 LLM 生成答案。</p>

1. Baseline
   - 將 demo 中的資料，替換成我們提供 or 自己的資料集
   - 能夠檢索相似資料
   - 基於檢索的資料進行回答
2. Advanced（Optional）
   - Embedding 怎麼儲存？每次都要重新計算嗎？
   - 該如何處理太久以前的歷史資料？
   - 利用 Gradio or Hugging Face Spaces 部署、分享 Chatbot

In [3]:
from pprint import pprint

from torch import cuda, bfloat16

from transformers import pipeline
from transformers import Conversation
conversation = Conversation() # 建立一個對話 Conversation 物件

from transformers import BitsAndBytesConfig, AutoConfig, AutoModelForCausalLM, AutoTokenizer
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
print(device)

from sentence_transformers import SentenceTransformer


embedding_model = SentenceTransformer("intfloat/multilingual-e5-large")

cuda:0


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

因為載入模型較大，使用 T4 GPU 時建議進行量化，以下程式為量化處理過程，</p>
在此先不贅述，有興趣的可以參考 Hugging face 官方文件～</p>

與前面範例不同的是，模型載入方法，我們透過 `AuToModelForCausalLM` 實例化模型，將其作為參數傳入 `pipeline`。

In [4]:
model_id = 'MediaTek-Research/Breeze-7B-32k-Instruct-v1_0'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

model_config = AutoConfig.from_pretrained(
    model_id
)

tokenizer = AutoTokenizer.from_pretrained(
    model_id)

hf_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto'
)

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.33k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/911k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.60G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/512M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [5]:
chatbot = pipeline(
    "text-generation",
    model=hf_model,
    tokenizer=tokenizer, # Tokenizer，要與模型匹配，主要提供 chat 模式時的特殊符號
    max_new_tokens=1024, # 模型最多可以生成多少字
    return_full_text=False # 控制 pipeline 只輸出 AI Message
)

Demo

In [6]:
# qa_data = [
#     """1. 入營注意事項：役男入營前無須先行理髮，待入營後統一安排理髮，再向役男收取費用
#     2. 役男可攜帶手機(不得為大陸廠牌，且不提供充電)，部隊會集中保管，於每日夜間以定時定點方式使用。
#     3. 營區不提供充電，建議攜帶拋棄式(手動)刮鬍刀；不可攜帶噴霧式液體。
#     4. 役男可配戴隱形眼鏡，但因基礎訓練期間生活緊湊，而隱形眼鏡需相當時間消毒清洗，建議配戴鏡片眼鏡為宜，並可多備一副眼鏡，以供替換。
#     """,

#     """報到入營時應該文件：
#     1. 徵集令。
#     2. 役男本人之國民身分證正本。
#     3. 私章。
#     4. 健保ＩＣ卡。
#     5. 郵局存摺正面影本。
#     6. 替代役役男輔導需求調查表。
#     7. 個人特殊醫療用品。
#     """,

#     """
#     替代役訓練天數
#     基礎訓練：21天(含撥交日)。(自253梯次起修正)
#     專業訓練：以各分發需用機關不同而有個不同期間的專業訓練。
#     """,

#     """
#     折抵役期規定
#     請備妥相關證明文件，如高中（職）以上各級學校經軍訓主管驗證加註折抵役期日數之成績單、大專集訓結訓證書、
#     驗退（停役）證明書或軍事學校退學（開除）證明書等正本（驗證後退還）。82年次以前出生者，
#     合計不得逾30日，83-93年次以後出生者，合計不得逾15日。已受軍事入伍訓練者，請於收到徵集令時，向戶籍地區公所提出免受基礎訓練申請。
#     """,

#     """
#     要當兵時，健保要辦理轉出轉入嗎?
#     即將入營服常備兵役之役男需持徵集令向全民健保加保單位辦理轉出，轉出日期填報「入營當月份」，轉入單位免填，後續將由國軍單位接續辦理後續轉入程序。
#     """,

#     """當兵期間義務役薪水的入帳戶一定要郵局嗎?
#     依現行規定目前有郵局、台新、土地銀行跟合作金庫這四家金融機構可使用。"""
# ]

In [12]:
qa_data = [
    """
    因普通傷害、疾病或生理原因必須治療或休養者，得依下列規定請普通傷病假。
    未住院者，一年內合計不得超過三十日。
    住院者，兩年內不得超過一年。
    未住院傷病假與住院傷病假兩年內合計不得超過一年。
    """,
    """
    員工陪伴其配偶妊娠產檢或其配偶分娩時，給予陪產檢及陪產假七日。
    除陪產檢於配偶妊娠請假外，員工陪產之請假，應以配偶分娩當日、及其前後合計三十日期間內為之，陪產檢及陪產假期間，工資照給。
    """,
    """
    外出怎麼申請
    1.路徑：集團人事系統>差勤管理 > 我的差勤 > 請假調整 > 外出申請
    2.輸入外出別、當天及之後日期、事由、代理人
    3.點選申請後主管審核
    4.回到辦公室後點選外出結束
    """,
    """
    請問特休假怎麼移到別天?
    路徑：集團人事系統>差勤管理>我的差勤>請假調整>請假申請/變更，點選欲調整之特休假日期選擇特休排假變更。
    """,
    """
    加班申請規則
    如因特定業務需要，得於正常工時外延長工時，惟每日延長之工作時間連同正常工作時間，一日不得超過十二小時、一個月不得超過四十六小時。員工自行要求延長工作時間，應經單位主管核准後始得為之。
    """,
    """
    福團理賠申請
    填寫申請書檢附診斷書正本，收據正本(如為副本需加蓋醫院章)及關係證明逕行向國泰人壽服務中心申請。
    路徑:
    產險EIP>文件管理>表單下載>管理部_人管科>福利
    https://eip.cathay-ins.com.tw/download?filePath=/okm:root/%E5%85%AC%E7%94%A8%E8%B3%87%E6%96%99%E5%A4%BE/%E8%A1%A8%E5%96%AE%E4%B8%8B%E8%BC%89/%E7%AE%A1%E7%90%86%E9%83%A8_%E4%BA%BA%E7%AE%A1%E7%A7%91/%E7%A6%8F%E5%88%A9%E9%A1%9E/%E5%9C%98%E9%AB%94%E4%BF%9D%E9%9A%AA%E7%90%86%E8%B3%A0%E7%94%B3%E8%AB%8B%E6%9B%B8(%E6%96%B0)11303.pdf
    """,
    """
    怎麼設定職務代理人
    路徑：集團人事系統>差勤管理 > 代理人 > 代理人維護 > 職務代理人設定 > 查詢> 設定代理人
    """
]

In [10]:
from typing import List
import numpy as np

def get_answer(query: str, source: List[str]):
    most_related_sentence = None
    max_similarity = 0

    for sentence in source:
        sim = calculate_cosine_similarity(
        embedding_model.encode(query),
        embedding_model.encode(sentence)
        )

        if sim > max_similarity:
            most_related_sentence = sentence
            max_similarity = sim

    return most_related_sentence

def calculate_cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [13]:
user_query = input(">>>")
conversation = Conversation()

while user_query.lower() != "bye":
    print(f"user: {user_query}")
    # 尋找最相似的文件
    answer = get_answer(user_query, qa_data)
    llm_input = f"""請你基於以下資訊回答使用者的問題
    {answer}
    ===
    問題：{user_query}
    """
    conversation.add_user_input(llm_input)
    # 將 conversation.messages 丟給 chatbot
    chatbot_result = chatbot(conversation.messages)[0]['generated_text']
    print(f"AI: {chatbot_result}")
    conversation.append_response(chatbot_result)

    user_query = input(">>>")


>>>產假
user: 產假
AI: 使用者的問題：產假

答案：

根據提供的資訊，員工在配偶妊娠期間可以申請陪產檢假，假日為七日。陪產檢及陪產假期間，工資照給。陪產檢及陪產假應於配偶分娩之當日以及前後合計30日之內使用。

在回答使用者的問題時，可以考慮以下重點：

1. 陪產檢及陪產假的天數：7天
2. 請假期間的限制：應於配偶分娩之當日以及前後合計30日之內使用
3. 工資待遇：陪產檢及陪產假期間，工資照給
>>>忘記打卡怎麼辦
user: 忘記打卡怎麼辦
AI: 使用者的問題：忘記打卡怎麼辦

答案：

如果忘記打卡，可以在集團人事系統中進行職務代理人的設定。首先，訪問差勤管理 > 代理人 > 代理人維護 > 職務代理人設定，然後選擇需要設定代理人的員工，在此頁面中，可以查詢已設定的代理人，也可以進行代理人設定。如果尚未設定代理人，可以在此處進行代理人選擇並確認設定。

在回答使用者的問題時，可以考慮以下重點：

1. 系統操作路徑：集團人事系統 > 差勤管理 > 代理人 > 代理人維護 > 職務代理人設定
2. 如何查詢代理人：在查詢頁面中可以找到已設定的代理人
3. 如何設定代理人：在職務代理人設定頁面中，選擇需要設定代理人的員工並確認設定
>>>bye
